In [147]:
import numpy as np
import math
from matplotlib import pyplot as plt
from PIL import Image

In [148]:
im = Image.open("TestImage2c.jpg")
print(im.format, im.size, im.mode)

('JPEG', (1008, 756), 'RGB')


In [149]:
col, row = im.size
data = np.zeros((row * col, 3))
pixels = im.load()
print col, row
for i in range(row):
    for j in range(col):
        r, g, b = pixels[j, i]
        data[col * i + j, :] = r, g, b
grayscale = np.zeros(row * col)
for i in range(row):
    for j in range(col):
        grayscale[col * i + j] = data[col * i + j, 0] * 0.3 + data[col * i + j, 1] * 0.59 + data[col * i + j, 2] * 0.11
grayscale.resize((row, col))

1008 756


In [67]:
im = Image.fromarray(grayscale)
im.show()
# greyscale done

In [150]:
sobel = np.zeros((row, col, 2))
magnitude = np.zeros((row, col))
thita = np.zeros((row, col))
# Gx, Gy, G, thita
max = 0
min = 9999

threshold = 20
for i in range(1, row - 1):
    for j in range(1, col - 1):
        sobel[i, j, 0] = grayscale[i - 1, j + 1] + 2 * grayscale[i, j + 1] + grayscale[i + 1, j + 1] \
                         - grayscale[i - 1, j - 1] - 2 * grayscale[i, j - 1] - grayscale[i + 1, j - 1]
        sobel[i, j, 1] = grayscale[i - 1, j - 1] + 2 * grayscale[i - 1, j] + grayscale[i - 1, j + 1] \
                         - grayscale[i + 1, j - 1] - 2 * grayscale[i + 1, j] - grayscale[i + 1, j + 1]
        magnitude[i, j] = (sobel[i, j, 0] ** 2 + sobel[i, j, 1] ** 2) ** 0.5
        if sobel[i, j, 0] == 0:
            thita[i, j] = math.atan(sobel[i, j, 1] / 0.01)
        else:
            thita[i, j] = math.atan(sobel[i, j, 1] / sobel[i, j, 0])
        if magnitude[i, j] > max:
            max = magnitude[i, j]
        if magnitude[i, j] < min:
            min = magnitude[i, j]
print max, min
count = 0
for i in range(1, row - 1):
    for j in range(1, col - 1):
        magnitude[i, j] = magnitude[i, j] / max * 255
        if magnitude[i, j] > threshold:
            magnitude[i, j] = 255
            count += 1
        else:
            magnitude[i, j] = 0
print count

952.571879598 0.0
32060


In [155]:
im = Image.fromarray(magnitude)
im.show()
# greyscale done

In [167]:
expand_old = np.copy(magnitude)
expand_new = np.copy(magnitude)
flag = 0
for k in range(3):
    for i in range(1, row - 1):
        for j in range(1, col - 1):
            flag = 0
            if expand_old[i-1,j]>0: flag += 1
            if expand_old[i+1,j]>0: flag += 1
            if expand_old[i,j-1]>0: flag += 1
            if expand_old[i,j+1]>0: flag += 1
            if flag>= 2:
                expand_new[i,j] = 255
    expand_old = np.copy(expand_new)

In [166]:
im = Image.fromarray(expand_new)
im.show()

In [168]:
distance0 = np.copy(expand_new)
for i in range(1,row-1):
    for j in range(1,col-1):
        if distance0[i,j]>0:
            distance0[i,j]=1
distance_new = np.copy(distance0)
distance_old = np.copy(distance0)
for k in range(15):
    for i in range(1,row-1):
        for j in range(1,col-1):
            min=999
            if distance_old[i-1,j]<min:
                min = distance_old[i-1,j]
            if distance_old[i+1,j]<min:
                min = distance_old[i+1,j]
            if distance_old[i,j-1]<min:
                min = distance_old[i,j-1]
            if distance_old[i,j+1]<min:
                min = distance_old[i,j+1]
            distance_new[i,j]=distance0[i,j]+min
    distance_old = np.copy(distance_new)

for i in range(1,row-1):
    for j in range(1,col-1):
        if distance_old[i,j]<distance_old[i-1,j] or distance_old[i,j]<distance_old[i+1,j] or \
        distance_old[i,j]<distance_old[i,j-1] or distance_old[i,j]<distance_old[i,j+1]:
            distance0[i,j] = 0

count = 0
for i in range(1,row-1):
    for j in range(1,col-1):
        if distance0[i,j] > 0:
            distance0[i,j]=255
            count +=1

In [172]:
im = Image.fromarray(distance0)
im.show()
# greyscale done

In [171]:
point = np.zeros((count, 2))
count = 0
for i in range(1, row - 1):
    for j in range(1, col - 1):
        if distance0[i, j] == 255:
            point[count, :] = i, j
            count += 1
count

9826

In [173]:
# hough transform
angle = 720
p = 4000
hough = np.zeros((angle, p))
for i in range(count):
    if (i % 1000)==0: print i
    for j in range(angle):
        x, y = point[i,:]
        tmpAngle = j / 720.0 * math.pi
        tmpP = x * math.cos(tmpAngle) + y * math.sin(tmpAngle)
        tmpP = int(tmpP) + 2000
        hough[j, tmpP] +=1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [174]:
threshold = 50
for i in range(angle):
    for j in range(p):
        if hough[i, j] > threshold:
            count += 1
print count

tmpHough = np.zeros((count, 3))
tmpCount = 0
for i in range(angle):
    for j in range(p):
        if hough[i, j] > threshold:
            tmpHough[tmpCount,:]=hough[i,j],i,j
            tmpCount += 1

print tmpCount
effectiveHough = np.zeros((count, 3))
count = 0
for i in range(tmpCount):
    tmp = tmpHough[i,0]
    tmpAngle=tmpHough[i,1]
    tmpP=tmpHough[i,2]
    top = 0 if tmpAngle - 3 < 0 else int(tmpAngle - 3)
    bottom = angle - 1 if tmpAngle + 3 > angle - 1 else int(tmpAngle + 3)
    left = 0 if tmpP - 20 < 0 else int(tmpP - 20)
    right = p - 1 if tmpP + 20 > p - 1 else int(tmpP + 20)
    maxFlag = 1       
    for k in range(top, bottom + 1):
            for l in range(left, right + 1):
                if tmp < hough[k, l]:
                    maxFlag = 0
    if maxFlag == 1:
        effectiveHough[count, :] = tmp, tmpAngle, tmpP
        count+=1
print count
    
# for i in range(0, angle):
#     for j in range(0, p):
#         tmp = hough[i, j]
#         top = 0 if i - 3 < 0 else i - 3
#         bottom = angle - 1 if i + 3 > angle - 1 else i + 3
#         left = 0 if j - 10 < 0 else j - 10
#         right = p - 1 if j + 10 > p - 1 else j + 10
#         maxFlag = 1
#         for k in range(top, bottom + 1):
#             for l in range(left, right + 1):
#                 if tmp < hough[k, l]:
#                     maxFlag = 0
#         if (tmp > threshold) and maxFlag == 1:
#             effectiveHough[count, :] = hough[i, j], i, j
#             count += 1
# print count
#get effectiveHough

# for i in range(100):
#     print effectiveHough[i,0]

for i in range(count):
    for j in range(i + 1, count):
        if effectiveHough[i, 0] < effectiveHough[j, 0]:
            tmpNum, tmpAngle, tmpP = effectiveHough[i, :]
            effectiveHough[i] = effectiveHough[j]
            effectiveHough[j] = tmpNum, tmpAngle, tmpP

#sort effectiveHough

11712
1886
150


In [138]:
for i in range(100):
    print effectiveHough[i]

[  403.   644.  1677.]
[  376.   645.  1574.]
[  376.   646.  1573.]
[  329.   142.  2838.]
[  311.   149.  2904.]
[  308.   689.  1744.]
[  288.   684.  1764.]
[  288.   222.  2709.]
[  288.   701.  1701.]
[  286.   657.  1902.]
[  276.   693.  1810.]
[  248.   167.  2330.]
[  239.   346.  2480.]
[  219.   714.  1644.]
[  218.   675.  1805.]
[  217.    97.  2426.]
[  214.   718.  1638.]
[  207.   710.  1663.]
[  202.     3.  2375.]
[  199.     8.  2392.]
[  196.    80.  2401.]
[  195.   532.  2000.]
[  194.   180.  2533.]
[  193.   560.  2000.]
[  191.   525.  2000.]
[  187.   122.  2443.]
[  186.   713.  1692.]
[  184.   649.  2000.]
[  183.   629.  2000.]
[  182.   717.  1685.]
[  182.     4.  2321.]
[  182.   719.  1688.]
[  179.   138.  2452.]
[  179.   139.  2454.]
[  179.   140.  2455.]
[  179.   520.  2000.]
[  178.    87.  2407.]
[  173.   605.  2000.]
[  172.   215.  2659.]
[  170.    63.  2394.]
[  169.   180.  2451.]
[  168.     9.  2336.]
[  168.    22.  2361.]
[  167.   6

In [200]:
resultImage = np.copy(magnitude)
for i in range(row):
    for j in range(col):
        for k in range(20):
            if int(i * math.cos(effectiveHough[k, 1] / 720.0 * math.pi) +
                                   j * math.sin(effectiveHough[k, 1] / 720.0 * math.pi)) - effectiveHough[
                k, 2] +2000 == 0:
                resultImage[i, j] = 255

In [201]:
im = Image.fromarray(resultImage)
im.show()

### effectiveHough 数组中的内容：
* 落在该直线上点的数量
* thita:角度，取值0-720
* p:直线到原点距离，取值-2000-2000

#### 该数组已sort

### 直线——线段
* 对图中所有点搜索
* effectiveLinePoint[100,max(effectiveHough[0]),2],如果点在前一百的hough直线上，加入到数组中
* 对数组作sort
* 求出数组中的线段

In [202]:
maxEffectvieHough = int(effectiveHough[0,0])
effectiveLinePoint = np.zeros((100,500,2))
lineCount = np.zeros(100)
for i in range(row):
    for j in range(col):
        if magnitude[i,j]==255:
            for k in range(100):
                onLine = int(i * math.cos(effectiveHough[k, 1] / 720.0 * math.pi) + 
                    j * math.sin(effectiveHough[k, 1] / 720.0 * math.pi)) - effectiveHough[k, 2] +2000 
                if onLine==0:
                    tmp = int(lineCount[k])
                    effectiveLinePoint[k,tmp,:]=i,j
                    lineCount[k]+=1
for i in range (20):
    print lineCount[i]

305.0
328.0
397.0
309.0
248.0
260.0
246.0
257.0
224.0
263.0
170.0
127.0
168.0
203.0
155.0
211.0
211.0
111.0
111.0
150.0


In [203]:
for i in range(100):
    count = int(lineCount[i])
    for j in range(count):
        for k in range(j+1,count):
            if effectiveLinePoint[i, j, 0] > effectiveLinePoint[i, k, 0]:
                tmpI, tmpJ = effectiveLinePoint[i, j, :]
                effectiveLinePoint[i, j, :] = effectiveLinePoint[i, k, :]
                effectiveLinePoint[i, k, :] = tmpI, tmpJ

In [141]:
effectiveHough[5,:]

array([  308.,   689.,  1744.])

In [120]:
for i in range(int(lineCount[5])):
    print effectiveLinePoint[5,i,:]

[ 365.  612.]
[ 367.  611.]
[ 368.  610.]
[ 370.  609.]
[ 371.  608.]
[ 372.  608.]
[ 373.  607.]
[ 374.  606.]
[ 375.  606.]
[ 376.  605.]
[ 377.  604.]
[ 378.  603.]
[ 378.  604.]
[ 379.  603.]
[ 380.  602.]
[ 381.  601.]
[ 381.  602.]
[ 382.  601.]
[ 383.  600.]
[ 384.  599.]
[ 384.  600.]
[ 385.  599.]
[ 386.  598.]
[ 387.  597.]
[ 388.  597.]
[ 389.  596.]
[ 390.  595.]
[ 391.  595.]
[ 392.  594.]
[ 393.  593.]
[ 394.  592.]
[ 394.  593.]
[ 395.  592.]
[ 396.  591.]
[ 397.  590.]
[ 397.  591.]
[ 398.  590.]
[ 399.  589.]
[ 400.  588.]
[ 400.  589.]
[ 401.  588.]
[ 402.  587.]
[ 403.  586.]
[ 404.  586.]
[ 405.  585.]
[ 406.  584.]
[ 407.  584.]
[ 408.  583.]
[ 409.  582.]
[ 410.  581.]
[ 410.  582.]
[ 411.  581.]
[ 412.  580.]
[ 413.  579.]
[ 413.  580.]
[ 414.  579.]
[ 415.  578.]
[ 416.  577.]
[ 416.  578.]
[ 417.  577.]
[ 418.  576.]
[ 419.  575.]
[ 420.  575.]
[ 421.  574.]
[ 422.  573.]
[ 423.  573.]
[ 424.  572.]
[ 425.  571.]
[ 426.  570.]
[ 426.  571.]
[ 427.  570.]
[ 428.

### 已有图片

In [197]:
houghLines = np.zeros((row, col))
for i in range(50):
    for j in range(int(lineCount[i])):
        x,y = effectiveLinePoint[i,j,:]
        houghLines[int(x),int(y)]=255

In [220]:
im = Image.fromarray(magnitude)
im.show()

In [226]:
im = Image.fromarray(distance0)
im.show()

In [101]:
im = Image.fromarray(resultImage)
im.show()

In [198]:
im = Image.fromarray(houghLines)
im.show()

### 线段化

In [216]:
threshold = 50
# thita,p; start point; end point
effectiveLine = np.zeros((1000,3,2))
pointCount = 0
index = 0
from IPython.core.debugger import Tracer

for i in range(100):
    pointCount = 0
    count = int(lineCount[i])
    startX, startY = effectiveLinePoint[i,0,:]
    oldX, oldY = startX, startY
    for j in range(1,count):
        newX, newY = effectiveLinePoint[i,j,:]
        distance = math.sqrt((oldX-newX) **2 + (oldY-newY) **2 )
        if distance<12:
            oldX,oldY = newX, newY
            pointCount += 1
            if j==count-1:
                #Tracer()()
                distance = math.sqrt((oldX-startX) **2 + (oldY-startY) **2 )
                if distance >= 60:
                    effectiveLine[index,0,:]=effectiveHough[i,1], effectiveHough[i,2]
                    effectiveLine[index,1,:]=startX,startY
                    effectiveLine[index,2,:]=oldX,oldY
                    index += 1
        else:
            #Tracer()()
            distance = math.sqrt((oldX-startX) **2 + (oldY-startY) **2)
            if distance >= 60:
                effectiveLine[index,0,:]=effectiveHough[i,1], effectiveHough[i,2]
                effectiveLine[index,1,:]=startX,startY
                effectiveLine[index,2,:]=oldX,oldY
                index += 1
            startX, startY = newX, newY
            oldX, oldY = startX, startY
            pointCount = 0
print index

55


In [217]:
for i in range(index):
    print effectiveLine[i,:]

[[  142.  2837.]
 [  446.   817.]
 [  621.   572.]]
[[  645.  1576.]
 [  480.    94.]
 [  602.   452.]]
[[  646.  1673.]
 [  372.    79.]
 [  497.   454.]]
[[  147.  2902.]
 [  477.   869.]
 [  662.   622.]]
[[  167.  2330.]
 [  184.   290.]
 [  319.   139.]]
[[  221.  2711.]
 [  367.   611.]
 [  603.   448.]]
[[  656.  1903.]
 [  185.   290.]
 [  256.   540.]]
[[  692.  1815.]
 [  282.   771.]
 [  308.   984.]]
[[  686.  1760.]
 [  356.   755.]
 [  376.   892.]]
[[  686.  1760.]
 [  378.   905.]
 [  389.   977.]]
[[  692.  1738.]
 [  317.   425.]
 [  324.   485.]]
[[  692.  1738.]
 [  357.   755.]
 [  383.   965.]]
[[  214.  2659.]
 [  279.   614.]
 [  454.   485.]]
[[  351.  2468.]
 [  493.   449.]
 [  603.   445.]]
[[  345.  2481.]
 [  493.   450.]
 [  582.   444.]]
[[  715.  1642.]
 [  377.   856.]
 [  380.   965.]]
[[  716.  1642.]
 [  373.   827.]
 [  375.   924.]]
[[  396.  2018.]
 [  373.    78.]
 [  482.    95.]]
[[  397.  2016.]
 [  374.    78.]
 [  484.    96.]]
[[  361.  24

In [218]:
lines = np.zeros((row, col))
for i in range(index):
    angle, p = effectiveLine[i,0,:]
    startX, startY=effectiveLine[i,1,:]
    endX, endY=effectiveLine[i,2,:]
    startX = int(startX)
    endX = int(endX)
    for j in range(startX+1,endX):
        for k in range(0,col):
            onLine = int(j * math.cos(angle / 720.0 * math.pi) + 
                    k * math.sin(angle / 720.0 * math.pi) - p) +2000
            if onLine == 0:
                lines[j,k]=255

In [219]:
im = Image.fromarray(lines)
im.show()